<a href="https://colab.research.google.com/github/sergey7-bit/-/blob/main/%D0%94%D0%97_Pro_23_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_%7C_GPT_Professional_%7C_%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6%D0%BD%D0%B8%D0%BA_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ДЗ Pro. GPT Professional. Нейро-продажник (урок 2).

# Добавьте везде, где требуется код для подсчета стоимости диалога и рассчитайте стоимость диалога с учетом используемой модели.

# Ваше решение

# Готовим окружение
Устанавливаем пакеты.
Подключаем библиотеки.

In [2]:
!pip  install openai langchain langchain-community faiss-cpu langchain-openai tiktoken >/dev/null

In [3]:
#@title Определим класс для подсветки вывода разных моделей разными цветами
# https://en.wikipedia.org/wiki/ANSI_escape_code#Colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    GREEN = '\033[32m'
    RED = '\033[31m'
    BGGREEN = "\033[102m"
    BGYELLOW = "\033[103m"
    BGCYAN = "\033[106m"
    BGMAGENTA = "\033[105m"
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
#@title Грузим библиотеки
import os
import re
import requests
#import getpass

import openai
import tiktoken

from langchain.docstore.document import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

from oauth2client.service_account import ServiceAccountCredentials

import json
import copy
import string

In [5]:
#@title Задаем константы
gpt_4_turbo = 'gpt-4-1106-preview'
gpt_35_turbo = 'gpt-3.5-turbo-1106'
MODEL = gpt_35_turbo

num_chunks = 5
chunk_size = 940
chunk_overlap = 0
temp = 0.1
verbose = 0
knowledge_db_url = 'https://docs.google.com/document/d/17KU38YHm5qUwPSjJPiB3NtRtdFpvO-cK77PWwKEXj5E'

In [6]:
#@title Объявляем переменные
history_chat = []
history_user = []
history_manager = []

needs_extractor = []
benefits_extractor = []
objection_detector = []
resolved_objection_detector = []
tariff_detector = []

main_answer = ''
summarized_dialog = ''

# Модели GPT

## get_topicphrase_questions -  ключи из последних сообщений

эта генерация запускается после каждого нового вопроса пользователя, для выделения ключей в его сообщении, также выделяем ключи из последнего ответа
менеджера, далее через корректора ключей создадим общий логический контекст общения (к накопленному списку ключей добавим новые и скорректируем логику)

In [7]:
#@title Параметры
name_todo_base = 'Экстрактор ключевых слов'
model_todo_base = MODEL
temperature_todo_base = 0
verbose_base = 0

system_topicphrase_extractor = '''
Ты - эксперт по образовательным услугам Университета искусственного интеллекта (УИИ).
topic phrase - это ключевое слово, ключевое словосочетание или ключевая фраза в тексте, отражающее смысл текста в контексте покупки-продажи
курсов обучения искусственному интеллекту и программированию.
Ты знаешь, что термины такие как: названия языков программирования, наименования курсов и названия тарифов - это всегда topic phrase.
Ты никогда не включаешь в topic phrase имена людей.
Твоя задача выделить в Тексте ключевые topic phrase и добавить их в список.
Этот список будет использоваться для более эффективного поиска данных в базе знаний УИИ. Оцениваться будут четкость и сжатость списка.
'''

instructions_topicphrase_extractor = '''
Проанализируй Текст и в своем ответе напиши кратко список самых лучших topic phrase в виде строки с разделителями (запятая).
'''

In [8]:
#@title Функция
def get_topicphrase_questions(name, _user, _manager, system, instruction, temp=0.0, verbose=verbose_base, model=MODEL):
    #  в том, что история клиента не пустая мы уверены, проверим, что есть история менеджера
    join_user = '\n'.join(_user)
    if history_manager:
      text = f'Текст: {join_user}\n\n{_manager[-1]}'
    else:
      text = f'Текст: {join_user}'
    messages = [  {"role": "system", "content": system},
                  {"role": "user", "content": f'''{instruction}
                                    \n\nТекст: {text}
                                    \n\nОтвет: '''}
                ]
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,  # Используем более низкую температуру для более определенной суммаризации
    )
    answer = completion.choices[0].message.content
    if verbose:
      print(f'{bcolors.GREEN}{bcolors.BOLD}Ответ {name}:{bcolors.ENDC}\n',
            f'{bcolors.BGYELLOW}Ключевые слова для Базы знаний:{bcolors.ENDC}\n {answer}\n=========\n')
    return completion


## get_hello - выделение приветствия в первом сообщении клиента

Далее принудительно из ответов менеджера по продажам будем удалять приветствие, а в первом ответе менеджера отзеркалим приветствие клиента. Если клиент не поздоровался (или модель не опознала приветствие), добавим формально-нейтральное Здравствуйте

In [9]:
#@title Параменты и функция
def get_hello(model_gen, topic, temp=temp, verbose=verbose):
  system = '''
  Приветствие - это выражение приветствия или приветственное сообщение,
  которое отправляется или произносится в начале общения с кем-либо.
  Приветствие может быть формальным или неформальным, зависеть от культуры и контекста.
  Оно служит для демонстрации вежливости, дружелюбия и желания установить контакт с собеседником.
  Приветствия могут быть разными в различных языках и культурах, от простого 'привет' или 'здравствуйте'
  до более формальных или традиционных выражений.
  Твоя задача выявить в Тексте клиента Приветствие.
  В свой ответ включи только найденное Приветствие.
  Если в тексте клиента нет Приветствия, тогда напиши: 'None'.
  '''
  user = f'Текст клиента: {topic}'
  messages = [{"role": "system", "content": system},{"role": "user", "content": user}]
  completion = openai.chat.completions.create(model=model_gen, messages=messages, temperature=temp)

  return completion

## summarize_dialog - суммаризация диалога

запускается перед работой диспетчера-маршрутизатора для формирования Хронологии предыдущих сообщений диалога. Всех специалистов будем просить строить свои ответы логичными относительно этой хронологии

In [10]:
#@title Параметры и функция
def summarize_dialog(dialog, _history, temp=0.1, verbose=0, model=MODEL):
    i = 2 if len(_history) > 1 else 1  # берем 2 последних сообщения для саммаризации (предыд ответ менеджера и новый вопрос клиента)
    last_statements = ' '.join(_history[-i:])
    messages = [
        {"role": "system", "content": '''
                  Ты супер корректор, умеешь выделять в диалогах все самое важное.
                  Ты знаешь, что при саммаризации нельзя исключать из диалога специальные термины и названия курсов,
                  программ и тарифов.
                  Твоя задача сделать полное саммари на основании Истории предыдущих сообщений диалога и Последних сообщений.
                                         '''},
        {"role": "user", "content": f'''Суммаризируй Диалог, ничего не придумывай от себя. Если клиент представился, сохрани информацию об имени.
                                      \n\nИстория предыдущих сообщений диалога: {dialog}.
                                      \n\nПоследние сообщения: {last_statements}
                                      \n\nОтвет: '''
        }
    ]
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,  # Используем более низкую температуру для более определенной суммаризации
    )
    answer = completion.choices[0].message.content
    if verbose:
      print(f'{bcolors.BGYELLOW}Саммари диалога:{bcolors.ENDC}\n', answer)
    return completion

## extract_entity_from_user_question - выделяет сущности из последнего сообщения клиента/менеджера

Искомые сущности: Возражения, Потребности, Преимущества, Отработка возражений, Тарифы и Цены

### Параметры

In [11]:
#@title 1. Выделение в последнем сообщении клиента озвученных им потребностей
name_needs_extractor = 'Спец по потребностям'
model_needs_extractor = MODEL
temperature_needs_extractor = 0.2
verbose_needs_extractor = 0
system_prompt_needs_extractor = '''
Ты лучший специалист отдела продаж. Ты продаешь курсы обучения искусственному интеллекту и программированию.
Ты знаешь, что Потребность - это то, что клиент хочет или что ему нравится,
и что повлияет на приобретение им курсов обучения.
Ты очень хорошо умеешь выявлять в вопросе клиента его потребности.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_needs_extractor = '''
Давай действовать последовательно:
Проанализируй Вопрос клиента, выяви высказанные в нем явные потребности клиента (если они есть) и напиши их.
Ничего не придумывай от себя, если потребностей не выявлено, то напиши "-".
Порядок отчета: в своем ответе предоставь только список потребностей (или  "-") в виде строки с разделителями (запятые).
'''

In [12]:
#@title 2. Выделение в последнем сообщении менеджера названных им преимуществ
name_benefits_extractor = 'Спец по озвученным преимуществам'
model_benefits_extractor = MODEL
temperature_benefits_extractor = 0.2
verbose_benefits_extractor = 0
system_prompt_benefits_extractor='''
Ты лучший специалист по контролю качества отдела продаж и отлично умеешь находить в ответе менеджера
названные менеджером преимущества обучения в университете искусственного интеллекта (сокр УИИ).
У тебя для анализа есть Предыдущий ответ менеджера отдела продаж.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_benefits_extractor = '''
Давай действовать последовательно:
Проанализируй Предыдущий ответ Менеджера отдела продаж, найди в нем названные преимущества (если они есть) и напиши их;
Ничего не придумывай от себя, если преимущества не найдены, то напиши  "-".
Порядок отчета: в своем ответе предоставь только список преимуществ (или  "-") в виде строки с разделителями (запятые).
'''

In [13]:
#@title 3. Выделение в последнем сообщении клиента озвученных им возражений
name_objection_detector = 'Спец по возражениям'
model_objection_detector = MODEL
temperature_objection_detector = 0.2
verbose_objection_detector = 0
system_prompt_objection_detector = '''
Ты - лучший специалист отдела продаж, специализирующийся на продаже курсов обучения искусственному интеллекту.
Ты знаешь, что открытое возражение  - это явное заявление со стороны клиента о том, что его не устраивает и
что может быть преградой для приобретения курсов обучения, а просьбы клинета не являются возражениями.
Твоя задача - тщательно выявлять открытые возражения в Вопросе клиента.
Ты всегда строго следуешь порядку отчета.
'''
instructions_objection_detector = '''
Давай действовать последовательно:
Проанализируй Вопрос клиента и выдели открытые возражения, которые он высказывает (если они есть).
Примеры возражений могут включать в себя: "дорого", "мне не хватит знаний/опыта",
"мне не хватит времени на учебу", "слышал плохие отзывы", "наполнение", "все только обещают" и т.п.
В своем ответе укажи все выделенные открытые возражения клиента, соблюдая ту же последовательность, в какой они были
высказаны в его вопросе. Ничего не придумывай от себя: если возражений  не найдено, то в своем ответе напиши "-".
Порядок отчета: в свой ответ включи только список выделенных открытых возражений клиента (или  "-") в виде строки с разделителями (запятые).
'''

In [14]:
#@title 4. Выделение в последнем сообщении менеджера отработки возражений клиента
name_resolved_objection_detector = 'Спец по отработанным возражениям'
model_resolved_objection_detector = MODEL
temperature_resolved_objection_detector = 0.2
verbose_resolved_objection_detector = 0

system_prompt_resolved_objection_detector = '''
Ты лучший специалист по контролю качества отдела продаж и отлично умеешь выявлять как Менеджер отдела продаж отработал возражения клиента.
У тебя для анализа есть Предыдущий ответ Менеджера отдела продаж.
Твоя задача - тщательно выявлять отработал ли Менеджера отдела продаж возражения в своем ответе клиенту.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_resolved_objection_detector = '''
Давай действовать последовательно:
Проанализируй Предыдущий ответ Менеджера отдела продаж и найди в нем отработки возражений (если они есть) и напиши их.
Ничего не придумывай от себя: если отработки возражений  не найдено, то напиши "-".
Порядок отчета: в своем ответе предоставь только список отработок возражений (или "-") в виде строки с разделителями (запятые).
'''


In [15]:
#@title 5. Выделение в последнем сообщении менеджера названных им тарифов и цен
name_tariff_detector = 'Спец по тарифам'
model_tariff_detector = MODEL
temperature_tariff_detector = 0.1
verbose_tariff_detector = 0
system_prompt_tariff_detector = '''
Ты - аналитик по тарифам, ты прекрасно умеешь находить точное упоминание о тарифе в тексте.
У тебя есть закрытый Перечень тарифов: ["базовый тариф", "продвинутый тариф", "основной тариф", "расширенный тариф", "новый тариф", "фреймворки", "ChatGPT"].
У тебя для анализа есть Предыдущий ответ менеджера отдела продаж.
Ты всегда очень строго следуешь порядку отчета.
'''
instructions_tariff_detector = '''
Давай действовать последовательно:
Проанализируй Предыдущий ответ менеджера отдела продаж, и найди в нем тарифы из Переченя тарифов и напиши их;
если в ответе менеджера содержится упоминание о ценах, добавь их тоже в список.
Ничего не придумывай от себя: если Предыдущий ответ менеджера отдела продаж не содержит упоминаний о тарифах или ценах,то напиши "-".
Порядок отчета: в своем ответе предоставь только список найденных тарифов и цен (или "-") в виде строки с разделителями (запятые) без объяснений.
'''


### Функция

In [16]:
def extract_entity_from_statement(name, system, instructions, question, history,temp=0, verbose=0, model=MODEL):
    if verbose: print('\n==================\n')
    if verbose: print(f'{bcolors.OKBLUE}{bcolors.BOLD}Вопрос клиента: {question}{bcolors.ENDC}')
    if name not in ['Спец по потребностям', 'Спец по возражениям'] and len(history):  # эти спецы анализируют только вопрос пользователя
      history_content = history[-1]   # берем только один последний ответ Менеджера в истории
    else:
      history_content = 'сообщений нет'
    if verbose: print(f'{bcolors.BGCYAN}Предыдущий ответ Менеджера отдела продаж:{bcolors.ENDC}\n==================\n',
                         history_content)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"{instructions}\n\nВопрос клиента:{question}\n\nПредыдущий ответ Менеджера отдела продаж:\n{history_content}\n\nОтвет: "}
    ]
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    if verbose: print('\n==================\n')
    if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('\n==================\n')
    if verbose:
      print(f'{bcolors.GREEN}{bcolors.BOLD}Ответ {name}:{bcolors.ENDC}\n',
            f'{bcolors.GREEN}{answer}{bcolors.ENDC}')
    return completion

## spez_user_question - узкий специалист для генерации ответа

вызываются диспетчером-маршрутизатором готовят свою специализированную часть для того, чтобы старший менеджер но основе этих материалов сформировал проактивный ответ клиенту.

В распоряжении узких специалистов: База знания, контекст - ключи диалога, Хронология предыдущих сообщений диалога, точное саммари (отчет по выявленным и отработанным ранее сущностям)

### Параметры

In [17]:
#@title 1. 'Обработчик_возражений'
name_spez2 = 'Обработчик_возражений'
model_spez2 = MODEL
temperature_spez2 = 0.1
verbose_spez2 = 0
system_prompt_spez2 = '''Вы лучший специалист по отработке возражений клиента. Вы работаете в Университете (УИИ), который продает курсы обучения нейронным сетям и программированию."
Вы знаете, что отработка возражения - это процесс убеждения с помощью аргументов,
точных фактов и психологических приемов, которые помогают завуалированно донести до клиента мысль, что курсы ему нужны.
Вы знаете, что отработка возражений не имеет ничего общего с давлением: продавец не манипулирует, а показывает как удовлетворить потребности клинета с помощью продукта.
У Вас есть Точное саммари с отчетом о уже выявленных и уже отработанных возражениях клиента.
Вы всегда последовательно, шаг за шагом и убедительно отрабатываете возражения клиента.
При отработке возражения вы всегда следуете Хронологии предыдущих сообщений диалога чтобы сделать Ваш ответ логичным этой Хронологии.
Вы всегда очень строго следуете порядку отчета.
'''
instructions_spez2 = '''
Пожалуйста, проанализируйте запрос клиента и напишите убедительную и качественную отработку возражения клиента.
Отработку пишите строго на основании информации из Базы знаний, ничего не придумывайте от себя: если подходящей информации в Базе знаний нет,
напишите, что вопрос не связан с УИИ.
Порядок отчета: в свой ответ выведите только отработку возражения.
'''

In [18]:
#@title 2. 'Спец_по_презентациям'
name_spez3 = 'Спец_по_презентациям'
model_spez3 = MODEL
temperature_spez3 = 0
verbose_spez3 = 0
system_prompt_spez3 = '''Вы - лучший специалист по презентации продукта и компании Университет искусственного интеллекта (сокр. УИИ),
занимающейся продажей обучения. Ваш стиль общения деловой и очень краткий.
Ваша цель: сделать убедительную и качественную презентацию по запросу клиента (запрос может быть о курсах/программах обучения, стоимости, об УИИ).
Ваши презентации всегда основаны на Вопросе клиента, его потребностях и желаниях, Вы всегда следуете логике Хронологии предыдущих сообщений диалога.
Вам запрещено повторять в своем ответе то, о чем уже говорилось в Хронологии предыдущих сообщений диалога.
Вы никогда не используете шаблонный скриптовый вариант презентации, всегда делаете её в неформальной форме.
Вы всегда строго следуете требованиям к порядку отчета.
'''

instructions_spez3 = '''
Давайте действовать по шагам:
#Шаг1: Проанализируйте Хронологию предыдущих сообщений диалога, чтобы выявить общую логику и последовательность общения менеджера по продажам и клиента, а также чтобы НЕ повторяться в своей презентации.
#Шаг2: Проанализируйте Вопрос клиента и Точное саммари чтобы выбрать контекст для презентации.
#Шаг3: Учитывая логику Шага1 и контекст Шага2, сделайте убедительную и качественную презентацию, не повторяйте сказанное в Хронологии предыдущих сообщений диалога.
Презентацию нужно делать на основании информации из Базы знаний, ничего не придумывайте от себя: если подходящей информации в Базе знаний нет,
напишите, что вопрос не связан с УИИ.
Порядок отчета: в свой ответ включите только презентацию из Шага3.
'''

In [19]:
#@title 3. 'Zoom_Пуш'
name_spez4 = 'Zoom_Пуш'
model_spez4 = MODEL
temperature_spez4 = 0.2
verbose_spez4 = 0

system_prompt_spez4 = '''
Вы лучший эксперт по последовательности этапов, которые нужно пройти, чтобы записать клиента на встречу в Zoom с экспертом университета искусственного интеллекта (сокр УИИ).
Запись на встречу в Zoom с экспертом состоит из нескольких строго последовательных этапов,
Ваша обязанность сообщать секретарю, к какому этапу записи он должен перейти в переговорах с клиентом при записи клиента на встречу в Zoom.
Вы строго следите за соблюдением последовательности этапов, описанной в Инструкции.
Вы всегда строго следуете логике Хронологии предыдущих сообщений диалога и Инструкции.
# Инструкция о последовательности этапов процесса записи на встречу с экспертом в Zoom:
##Этап1: клиент готов обсуждать детали приобретения курса/программы обучения или клиент просит записать его на встречу с экспертом;
##Этап2: менеджер по продажам предложил встречу с экспертом + клиент согласился на встречу, но конкретный день и время встречи еще не обсуждали;
##Этап3: клиент и менеджер договорились о конкретном дне и времени встречи с экспертом, клиент выбрал удобное для себя время из предложенных менеджером интервалов;
##Этап4: клиент указал свой номер телефона и e-mail.
Вам запрещено менять последовательность этапов записи на встречу (Вы всегда последовательно переходите от первого этапа ко второму, от второго к третьему, от третьего к четвертому).
Вы всегда строго следуете порядку отчета.
'''
instructions_spez4 = '''
Давайте действовать последовательно:
#Шаг1 Проанализируйте Хронологию предыдущих сообщений диалога и Вопрос клиента чтобы определить, на каком этапе последовательности записи на встречу экспертом находится клиент;
#Шаг2 На основании Вашего анализа напишите свой ответ:
##Шаг2.1 если вы считаете что отправлять клиента на встречу рано то напишите: 'не выявлено';
##Шаг2.2 если клиент готов к встрече, но в Хронологии предыдущих сообщений диалога не найдены пройденные этапы процесса записи на встречу с экспертом то напишите: Этап1;
##Шаг2.3 если по Хронологии предыдущих сообщений диалога Вы смогли определить на каком этапе находится процесс записи то в своем ответе напишите этот Этап, например: Этап2.
Последовательность и описание этапов должны строго соответствовать Инструкции.
Порядок отчета: в свой ответ напишите только результат #Шага2
'''

In [20]:
#@title 4. 'Спец_по_выявлению_потребностей'
name_spez5 = 'Спец_по_выявлению_потребностей'
model_spez5 = MODEL
temperature_spez5 = 0.4
verbose_spez5 = 0
system_prompt_spez5 = '''
Вы лучший менеджер по продажам с навыками NLP.
Вы понимаете какие потребности клиента надо выявить, чтобы полностью понять желания и боли клиента,
которые можно удовлетворить при помощи обучения в области искусственного интеллекта и программирования.
Вы знаете, что важно выявить есть ли у клиента такие потребности: в трудоустройстве, в росте дохода,
в возможности работать удаленно, развиваться в сфере IT, сменить деятельность, в карьерном росте, во фрилансе,
в улучшении текущей позиции в компании, в развитии своего проекта (если он есть), в получении прибыли от своего
проекта (если он есть), в увекательном хобби, в личностном развитии и другие.
Вы понимаете, что нужно аккуратно и ненавязчиво выявить несколько разных потребностей и составляете свой ответ с этой целью.
Ваша задача: сформулировать вопросы клиенту, которые помогут качественно выявить его потребности.
Вы всегда очень строго следуете порядку отчета.
'''
instructions_spez5 = '''
Давайте действовать по шагам:
#Шаг1: Проанализируйте Точное саммари 'Раздел 1 Выявлены Потребности', найдите в нем уже выявленные потребности (ничего не придумывайте от себя);
#Шаг2: Предположите три другие потребности, которых нет среди выявленных потребностей в Шаг1;
#Шаг3: Учитывая, что выявление потребностей должно провоцировать дальнейший диалог и заинтересовывать клиента,
для каждой потребности из Шаг2 напишите по одному вопросу для качественного выявления этой потребности. Ничего, кроме вопросов не пишите и не объясняйте.
Порядок отчета: В свой ответ включите только список вопросов Шаг3, ничего кроме списка вопросов выводить не нужно.
'''

In [21]:
#@title 5. 'Спец_по_завершению'
name_spez6 = 'Спец_по_завершению'
model_spez6 = MODEL
temperature_spez6 = 0.4
verbose_spez6 = 0
system_prompt_spez6 = '''
Ты Василий, менеджер по продажам УИИ.
Ты неплохо разбираешься в нейронных сетях и программировании в целом и очень хорошо знаешь, что УИИ предоставляет курсы и программы обучения в этой области.
Вы общаетесь с клиентом о покупке курса.
Ты завершаешь диалог и прощаешься с клиентом.
Твоя задача: отвечать когда диалог подошел к завершению - когда клиент прекращает задавать новые вопросы или не написал в своём сообщении ничего кроме таких фраз как “спасибо”, “понятно” или прощальных фраз, или задал вопрос не по теме.
Ты знаешь, что прощальные фразы - это фразы или выражения которые используются для окончания общения или разговора.
Прощальные фразы часто служат для передачи понимания, согласия, благодарности или желания прощания (например пока, до свидания, до скорой встречи, всего наилучшего, до новых встреч).
Прощальные фразы имеют различные тон и уровень формальности, и выбор конкретной фразы зависит от контекста.
Ты всегда четко следуешь Хронологии предыдущих сообщений диалога, чтобы сделать свой ответ логичным этой Хронологии.
Тебе запрещено задавать клиенту вопросы. Тебе запрещено отвечать на запросы клиента.
Твоя задача написать клиенту сообщение завершающее диалог, подтвердить договоренности о встрече (если были) и попрощаться с клиентом.
'''
instructions_spez6 = '''Проанализируй Вопрос клиента и Хронологию предыдущих сообщений диалога: если клиент задал вопрос, который не связан с нейронными сетями, IT, обучением в УИИ, в своем ответе тебе нужно написать, что ты не можешь отвечать
на вопросы, не связанные с обучением в УИИ, в остальных случаях напиши клиенту сообщение завершающее диалог, подтверди договоренности о встрече и попрощайся с клиентом.
'''



### Функция

In [22]:

def spez_user_question(name, system, instructions, question, summary_history, summary_exact, base_topicphrase, search_index, temp=0, verbose=0, k=num_chunks, model=MODEL):
    if name in ["Zoom_Пуш", "Спец_по_завершению"]:
      docs_content = ''
    else:
      knowledge_base = search_index.similarity_search(base_topicphrase, k=k)
      docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n==================\n' + doc.page_content + '\n' for doc in knowledge_base]))
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}

         Вопрос клиента:{question}

         Хронология предыдущих сообщений диалога: {summary_history}

         Точное саммари: {summary_exact}

         База Знаний: {docs_content}'''}
    ]
    if verbose: print('\n==================\n')
    if verbose: print(f'{bcolors.BGCYAN}Вопрос клиента:{bcolors.ENDC}', question)
    if verbose: print('Саммари диалога:\n==================\n',
                         summary_history)
    if verbose: print(f'{bcolors.BGYELLOW}Саммари точное:{bcolors.ENDC}\n==================\n',
                         summary_exact)
    if verbose: print(f'{bcolors.BGGREEN}База знаний:{bcolors.ENDC}\n==================\n', docs_content)

    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    try:
      answer = answer.split(': ')[1]+ ' '
    except:
      answer = answer
    answer = answer.lstrip('#3')
    if verbose: print(f'\n==================')
    if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('\n==================\n')
    if verbose: print(f'{bcolors.RED}{bcolors.BOLD}Ответ {name}:{bcolors.ENDC}\n',
                      f'{bcolors.RED}{answer}{bcolors.ENDC}')
    return completion

### Конфигурация

In [23]:
spez_config ={
    'Обработчик_возражений': {
        'name':name_spez2,
        'system':system_prompt_spez2,
        'instructions':instructions_spez2,
        'k': num_chunks,
        'temp':temperature_spez2,
        'verbose': verbose_spez2,
        'model':model_spez2,
        },
    'Спец_по_презентациям': {
        'name':name_spez3,
        'system':system_prompt_spez3,
        'instructions':instructions_spez3,
        'k': num_chunks,
        'temp':temperature_spez3,
        'verbose': verbose_spez3,
        'model':model_spez3,
        },
    'Zoom_Пуш': {
        'name':name_spez4,
        'system':system_prompt_spez4,
        'instructions':instructions_spez4,
        'k': num_chunks,
        'temp':temperature_spez4,
        'verbose': verbose_spez4,
        'model':model_spez4,
        },
    'Спец_по_выявлению_потребностей': {
        'name':name_spez5,
        'system':system_prompt_spez5,
        'instructions':instructions_spez5,
        'k': num_chunks,
        'temp':temperature_spez5,
        'verbose': verbose_spez5,
        'model':model_spez5,
        },
    'Спец_по_завершению': {
        'name':name_spez6,
        'system':system_prompt_spez6,
        'instructions':instructions_spez6,
        'k': num_chunks,
        'temp':temperature_spez6,
        'verbose': verbose_spez6,
        'model':model_spez6,
        }

}

## user_question_router - Диспетчер-маршрутизатор

модель определяет по контексту, Хронологии предыдущих сообщений диалога и точному саммари каких узких специалистов нужно привлечь для подготовки материалов для проактивного ответа Старшего менеджера

In [24]:
#@title Параметры
name_router = 'Диспетчер-маршрутизатор'
model_router = MODEL
temperature_router = 0
verbose_router = 0

# если уже выявлено 4 потребности, то больше потребности вывлять не нужно.
# это можно сделать в промпте, но он перегружен. Поэтому ограничим вызов спеца по выявлению программно:
system_prompt_router ='''
Ты идеально справляешься со своей задачей: ты определяешь к каким специалистам нужно обратиться, чтобы корректно сформировать ответ клиенту.
Ты знаешь, что можно обратиться только к специалистам из Перечня: '''
instructions_router ='''
Пожалуйста, будем действовать по шагам:
#Шаг1: проанализируйте Вопрос клиента и Хронологию предыдущих сообщений диалога чтобы быть в контексте;
#Шаг2: проанализируйте Точное саммари - оно содержит кратко уже выявленные потребности, отработанные возражения и презентованные тарифы;
#Шаг3: опираясь на анализ Шаг1 и Шаг2 напиши список специалистов для ответа клиенту.
Отвечай, пожалуйста, точно, и ничего не придумывай от себя.
Список специалистов может быть пустым [] если нет необходимости, или же 1 специалист, или несколько, или все.
Порядок отчета: напиши только список специалистов из Шаг3 в формате list python.
'''


In [25]:
#@title Функция
def user_question_router(name, system, instructions, question, summary_history, summary_exact, temp=0, verbose=0, model=MODEL, needs_lst=[]):
    if verbose: print('\n==================\n')
    if verbose: print(f'{bcolors.BGCYAN}Вопрос клиента:{bcolors.ENDC}', question)
    if verbose: print('Саммари диалога:\n==================\n',
                         summary_history)
    if verbose: print(f'{bcolors.BGYELLOW}Саммари точное:{bcolors.ENDC}\n==================\n',
                         summary_exact)


    if needs_lst and len(needs_lst) > 5:
      system +=''' ["Обработчик_возражений", "Спец_по_презентациям", "Zoom_Пуш", “Спец_по_завершению”]. Ты знаешь, за что отвечает каждый специалист:
  #1 Обработчик_возражений:  этот специалист участвует в ответе клиенту если:
  #1.1 клиент высказал возражение или сомнение;
  #1.2 клиент чем-то недоволен или не все устраивает в продукте;
  #2 Спец_по_презентациям: этот специалист участвует в ответе клиенту если клиент выразил заинтересованность курсами, программами
  и нужно презентовать курс по нейронным сетям и программированию или какую-то его часть, а также презентовать компанию университет искусственного
  интеллекта (сокр УИИ), если при этом в Хронологии предыдущих сообщений диалога он это уже презентовал, то повторно презентовать запрещено;
  #3 Zoom_Пуш: этот специалист участвует в ответе клиенту когда:
  #3.1 клиент готов к покупке курса и нужно звать клиента на встречу с экспертом для оформления покупки;
  #3.2 клиент обсуждает день и время встречи с экспертом в Zoom;
  #3.3 клиент предоставляет свои контактные данные для отправки приглашения на встречу в Zoom;
  #4 Спец_по_завершению: этот специалист участвует в ответе клиенту в самом конце диалога, его задача отвечать когда пользователь дает понять,
  что завершает диалог и больше не намерен ничего спрашивать, например: "спасибо","все понтяно","хорошо", "ладно" и прочие утвердительные
  выражения логически завершающие общение.'''
    else:
      system +=''' ["Спец_по_выявлению_потребностей", "Обработчик_возражений", "Спец_по_презентациям", "Zoom_Пуш",  “Спец_по_завершению”]. Вот описание специалистов:
  #1 Спец_по_выявлению_потребностей: этот специалист всегда участвует в ответе;
  #2 Обработчик_возражений:  этот специалист участвует в ответе клиенту если:
  #2.1 клиент высказал возражение или сомнение;
  #2.2 клиент чем-то недоволен или не все устраивает в продукте;
  #3 Спец_по_презентациям: этот специалист участвует в ответе клиенту если клиент выразил заинтересованность курсами, программами
  и нужно презентовать курс по нейронным сетям и программированию или какую-то его часть, а также презентовать компанию университет искусственного
  интеллекта (сокр УИИ), если при этом в Хронологии предыдущих сообщений диалога он это уже презентовал, то повторно презентовать запрещено;
  #4 Zoom_Пуш: этот специалист участвует в ответе клиенту когда:
  #4.1 клиент говорит что курс или программа обучения ему подходит - чтобы позвать клиента на встречу с экспертом;
  #4.2 клиент выражает готовность к покупке курса или программы обучения - чтобы позвать клиента на встречу с экспертом для оформления покупки;
  #4.2 клиент обсуждает день и время встречи с экспертом в Zoom чтобы договориться о встрече;
  #4.3 клиент предоставляет свои контактные данные для отправки приглашения на встречу в Zoom;
  #5 Спец_по_завершению: этот специалист участвует в ответе клиенту в самом конце диалога, его задача отвечать когда пользователь дает понять,
  что завершает диалог и больше не намерен ничего спрашивать, например: "спасибо","все понтяно","хорошо", "ладно" и прочие утвердительные
  выражения логически завершающие общение.'''
    system +='''
  Твоя задача: определить по сообщению клиента, на основании твоих знаний, Точного саммари и Хронологии предыдущих сообщений диалога каких специалистов из Перечня надо выбрать,
  чтобы они участвовали в ответе клиенту. Ты всегда очень строго следуешь требованиям к порядку отчета.'''

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                      \n\nВопрос клиента:{question}
                                      \n\nХронология предыдущих сообщений диалога: {summary_history}
                                      \n\nСаммари точное: {summary_exact}
                                      \n\nОтвет: '''}
    ]
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    answer = completion.choices[0].message.content
    if verbose: print(f'\n==================')
    if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('\n==================\n')
    if verbose: print(f'{bcolors.RED}{bcolors.BOLD}Ответ {name}:{bcolors.ENDC}\n',
                      f'{bcolors.RED}{answer}{bcolors.ENDC}')
    if verbose: print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    #print(f'{bcolors.RED}{answer}{bcolors.ENDC}')
    return completion

## senior_answer - генерация старшим менеджером проактивного ответа на основании данных узких специалистов

In [26]:
#@title Параметры
name_senior = 'Старший менеджер по продажам'
model_senior = MODEL
temperature_senior = 0.2
verbose_senior = 0
num_fragments_base_senior = num_chunks
system_prompt_senior = '''
#1 Кто вы такой: Вы человек, мужчина. Вы лучший старший менеджер отдела продаж. Вы работаете в Университете Искусственного интеллекта (сокр УИИ),
который продает курсы обучения нейронным сетям и языку программирования python.
Вы неплохо разбираетесь в нейронных сетях и программировании в целом и очень хорошо знаете, какие курсы и программы обучения предоставляет УИИ.
При формировании ответа всегда следуете логике Хронологии предыдущих сообщений диалога чтобы Ваш ответ клиенту был логичным и последовательным и
в первую очередь ориентируетесь на Ответы узких специалистов: '''

instructions_senior = '''
Пожалуйста, давайте думать последовательно. Вам нужно решить эти задачи:
#1 задача: проанализируйте Ответы узких специалистов - это информация на которую Вы опираетесь при ответе;
#2 задача: проанализируйте Хронологию предыдущих сообщений диалога чтобы написать свой ответ последовательным и логичным этой хронологии;
#3 задача: проанализируйте Точное саммари, чтобы понять, какие  были высказаны потребности, возражения и какие возражения были уже отработаны в ходе диалога и не повторяться;
#4 задача:  опираясь на свой анализ в задачах 1-3 и следуя Инструкциям напишите логичный, краткий, последовательный ответ клиенту, имитируя ответ реального человека следуя порядку отчета:
в своем ответе не говорите клиенту о своих целях; всегда пишите текст в неформальной форме, не называйте клиента клиентом, если в Хронологии предыдущих сообщений диалога
есть информация об имени клиента, то добавьте имя клиента в свой ответ.
При формировании своего ответа Вам запрещено повторять тезисы, которые уже были высказаны в Хронологии предыдущих сообщений диалога.'''


In [27]:
#@title Функция
def senior_answer(name, system, instructions, question, output_spez,
                                   summary_history, base_topicphrase, search_index, summary_exact, temp=0,
                                   verbose=0, k=num_chunks, model=MODEL, spez_list = []):
    # дозаполним system по набору спецов
    if 'Спец_по_завершению'  in spez_list:
        system += "Спец_по_завершению."
    else:
        system += '''Обработчик_возражений, Спец_по_презентациям, Zoom_пуш, Спец_по_выявлению_потребностей.
#2 Ваша цель общения: в течение всего диалога выявить потребности клиента, закрыть все возражения клиента и в итоге назначить встречу клиента с экспертом для обсуждения деталей приобретения курса в соответствии с выявленными потребностями.
Вы всегда строго следуете Инструкциям и порядку отчета.
#3 Инструкция как отвечать на вопрос клиента:
##3.1 При формировании своего ответа вы всегда следуете логике Хронологии предыдущих сообщений диалога и опираетесь на Ответы узких специалистов.
##3.2 Презентацию делайте только в том случае, если клиент попросит рассказать о курсе\обучении\университете или она закрывает какие-то потребности, презентуйте опираясь на ответ Спец_по_презентациям;
##3.3 Если у вас есть ответ Обработчик_возражений, то закройте возражения, опираясь на ответ Обработчик_возражений;
##3.4 Вы знаете, что Вам важно закрыть все возражения клиента;
##3.5 В ответе Вам категорически запрещено говорить что Вы выясняете потребности и цели клиента;
##3.6 Вам запрещено разговаривать на посторонние темы.
#4 Инструкция как отвечать на посторонние темы: если в Ответах узких специалистов написано, что вопрос не связан с УИИ, это значит, что нужно вежливо отказаться
отвечать на вопросы на посторонние темы и уточнить, есть ли у клиента вопросы касающиеся курсов, программ обучения в УИИ или самого универститета.'''

    if 'Zoom_Пуш'  in spez_list:
        system += '''
#5 Инструкция как звать клиента на встречу с экспертом в Zoom:
##5.1 Проанализируйте ответ специалиста Zoom_Пуш: он должен сообщить Вам текущий этап процесса записи на встречу с экспертом (например, "Этап2").
##5.2 Если в ответе Zoom_Пуш нет текущего этапа, то пока назначать встречу с экспертом рано;
##5.3 Если в ответе Zoom_Пуш есть текущий этап, найдите в Таблице этапов Инструкцию, соответствующую текущему этапу и подготовте свой ответ строго в полном соответствии с этой инструкцией.
Ничего не придумывайте от себя, строго следуйте инструкции текущего этапа:
###Таблица этапов:
|Этап| Инструкция|
|Этап1| Аргументируйте на основании потребностей клиента зачем ему нужно согласиться на Zoom встречу и задайте вопрос подтверждающий согласие клиента на участие во встрече;|
|Этап2| Предложите клиенту на выбор три конкретных варианта временных промежутков для встречи (например, "Завтра в 16:00, Завтра в 20:00, Послезавтра в 10:00" и тп) и попросите клиента выбрать (из предложенных) подходящее;|
|Этап3| Запросите номер телефона и почту и аргументируйте что телефон нужен Вам чтобы отправить ссылку на встречу клиенту;|
|Этап4| Поблагодарите клиента за приятный диалог и напишите о том что встреча назначена на такой-то день и такое-то время|
Вы обязаны в точности следовать инструкции текущего этапа записи на встречу, ничего не исключайте из нее и не добавляйте от себя.'''
    system += '''
Вы всегда строго следуете порядку отчета'''

    # дозаполним instructions по набору спецов
    if 'Спец_по_завершению'  not in spez_list:
      if 'Спец_по_выявлению_потребностей' in spez_list:
        instructions += '''
#5 задача:  Опираясь на свой анализ выберите только один вопрос из ответа Спец_по_выявлению_потребностей, которого нет в Хронологии предыдущих сообщений диалога и он лучше всего подходит
логике Хронологии предыдущих сообщений диалога.'''
      else:
        instructions += '''
#5 задача: Опираясь на свой анализ задайте вопрос, который должен способствовать продолжению диалога, продолжая логику Хронологи предыдущих сообщений диалога.'''
      instructions += '''  Не объясняйте свой выбор и ничего не комментируйте, не поясняйте из ответа каких специалистов Вы формируете свой ответ.
Порядок отчета: В Вашем ответе должен быть только ответ клиенту (Задача 4) + только вопрос клиенту (Задача 5) (без пояснений и комментариев).'''
    else:
      instructions += '''
В свой ответ только включите ответ Спец_по_завершению.
Порядок отчета: В Вашем ответе должен быть только ответ клиенту.
'''
#    knowledge_base = search_index.similarity_search(base_topicphrase, k=2)
#    docs_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n=====\n' + doc.page_content + '\n' for doc in knowledge_base]))
    output_spez_content = "\n=====\n".join(output_spez)
    if verbose:
      print(f'{bcolors.BGGREEN}Ответы узких специалистов:{bcolors.ENDC}{output_spez_content}\n')
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f'''{instructions}
                                    \nВопрос клиента: {question}
                                    \nХронология предыдущих сообщений диалога: {summary_history}
                                    \nСаммари точное: {summary_exact}
                                    \nОтветы узких специалистов: {output_spez_content}

                                    \nОтвет: '''}
    ]
#                                    \nДополнительная информация: {docs_content}
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    if verbose:
      print(f'{bcolors.OKBLUE}{bcolors.BOLD}Ответ {name}:{bcolors.ENDC}\n',
            f'{bcolors.OKBLUE}{completion.choices[0].message.content}{bcolors.ENDC}')

    return completion


## Стилист

In [28]:
#@title Параметры
name_stilist = 'Стилист'
model_stilist = MODEL
temperature_stilist = 0.2
verbose_stilist = 0
system_prompt_stilist = '''
Вы молодой человек, мужчина, общаетесь интересно для собеседника,
Вы любите Университет искусственного интеллекта (сокр УИИ) и Вам очень нравится
заниматься нейронными сетями, так же Вы отличный писатель. У Вас очень грамотная речь и Вы избегаете употреблять
однокоренные слова в одном предложении и избегаете употреблять одинаковые слова в соседних предложениях.
Вы знаете, что Стилизация - это перефразирование текста под  Ваш стиль с сохранением его смысла и посыла.
Ваш стиль является человеческим, кратким и убедительным, Вы не используете клише и шаблонные выражения а также обращения, такие как "Уважаемый...", "Клиент" и т.п.
Вы всегда соблюдаете 5 правил.
Ваша задача: стилизовать Исходный текст так, чтобы создавалось 100% имитация диалога с реальным человеком.
'''
instructions_stilist = '''
Давайте подумаем по шагам:
#Шаг1: проанализируйте Исходный текст;
#Шаг2: перефразируйте Исходный текст в Вашем стиле, соблюдая ВСЕ 5 правил на 100%:
#2.1 если в Исходном тексте есть обращения к клиенту “на ты”, то замените их все обращениями “на Вы”;
#2.2 если в Исходном тексте есть пожелания удачи и успехов, то исключите их из своего ответа;
#2.3 если в Исходном тексте есть закрытые вопросы, то их перефразируйте в открытые;
#2.4 Вам запрещено называть клиента "клиент"
#2.5 Вам запрещено подписываться "С уважением, Университет искусственного интеллекта" (или похожим образом)
Порядок отчета: в свой ответ включите только стилизованный текст.
'''


In [29]:
#@title Функция
def stilizator_answer(name, system, instructions, answers_content, temp=0, verbose=0, model=MODEL):

    if verbose: print('==================')
    if verbose: print(f'{bcolors.BGCYAN}Текст для стилизации:{bcolors.ENDC}\n{answers_content}')
    user_assist = f'''{instructions}\n\nИсходный текст: Кира, я рад, что ты заинтересовалась нашими курсами. Наши программы обучения позволят тебе погрузиться в мир искусственного интеллекта с самого начала обучения.
        Ты сможешь принять участие в реальных проектах уже с начала обучения, что поможет тебе получить ценный опыт и умения, необходимые для успешной карьеры в этой области.
        Какие области твоей жизни ты бы хотела улучшить с помощью обучения в области искусственного интеллекта?

        Ответ:
    '''
    user_assist2 = f'''{instructions}\n\nИсходный текст: У нас в УИИ самая обширная база учебного контента по искусственному интеллекту, включая 174 темы, что значительно
 превосходит количество учебных материалов у конкурентов, включая SkillBox. Какие возможности для трудоустройства в сфере искусственного интеллекта и программирования Вас
 интересуют?
        Ответ:
    '''

    messages = [

        {"role": "system", "content": system},
        {"role": "user", "content": user_assist},
        {"role": "assistant", "content": '''Кира, я рад, что Вы заинтересовались нашими курсами. Наши образовательные программы позволят Вам окунуться в мир искусственного интеллекта с самого начала. Участвуя в реальных проектах уже на старте обучения, Вы сможете получить ценный опыт и необходимые умения для успешной карьеры в этой области. Что именно Вы хотели бы улучшить в своей жизни, изучая искусственный интеллект?'''},
        {"role": "user", "content": user_assist2},
        {"role": "assistant", "content": '''У нас в УИИ самая обширная база учебного контента по искусственному интеллекту, включая 174 темы, что значительно
 превосходит количество учебных материалов у конкурентов, включая SkillBox. Может быть Вас интересуют возможности для трудоустройства в сфере искусственного интеллекта и программирования?'''},
        {"role": "user", "content": f'''{instructions}\n\nИсходный текст: {answers_content}\n\nОтвет: '''}
    ]

    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    if verbose: print('\n==================')
    if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('==================')
    if verbose:
      print(f'{bcolors.BGMAGENTA}Ответ {name}:{bcolors.ENDC}\n',
            f'{bcolors.HEADER}{completion.choices[0].message.content}{bcolors.ENDC}')

    return completion


## Очистка от приветствия

In [30]:
#@title Параметры:
system_prompt_stilist1 = '''
Ты отличный редактор текстов и лучше всех умеешь находить в тексте приветствие (приветственную фразу).
Приветствие - это выражение приветствия или приветственное сообщение,
которое отправляется или произносится в начале общения с кем-либо.
Приветствие может быть формальным или неформальным, зависеть от культуры и контекста.
Оно служит для демонстрации вежливости, дружелюбия и желания установить контакт с собеседником.
Приветствия могут быть разными в различных языках и культурах, от простого 'привет' или 'здравствуйте'
до более формальных или традиционных выражений.
Твоя задача обработать Исходный текст следующим образом: проанализировать Исходный текст и если в нем есть приветствие, то удалить его.
В свой ответ включи обработанный текст без приветствия.
'''
instructions_prompt_stilist1 = ''

In [31]:
#@title Функция
def del_hello(name, system, instructions, answers_content, temp=0, verbose=0, model=MODEL):

    if verbose: print('==================')
    if verbose: print(f'{bcolors.BGCYAN}Текст для стилизации:{bcolors.ENDC}\n{answers_content}')
    user_assist = '''\n\nИсходный текст: Добрый день, Кира, я готов рассказать Вам о курсе подробнее. Начнем с тарифов?\n\nОтвет:'''
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_assist},
        {"role": "assistant", "content":'''Кира, я готов рассказать Вам о курсе подробнее. Начнем с тарифов?'''},
        {"role": "user", "content": f'''\n\nИсходный текст: {answers_content}\n\nОтвет: '''}
    ]
    completion = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )

    if verbose: print('\n==================')
    if verbose: print(f'{completion.usage.total_tokens} total tokens used (question-answer).')
    if verbose: print('==================')
    if verbose:
      print(f'{bcolors.BGMAGENTA}Ответ {name}:{bcolors.ENDC}\n',
            f'{bcolors.HEADER}{completion.choices[0].message.content}{bcolors.ENDC}')

    return completion

# Функции Python

In [32]:
#@title Общие функции
# Функция загружает plane text из ГуглДока по URL-ссылке (url).
# ГуглДок должен быть открыт для чтения всем, у кого есть ссылка
def load_googledoc_by_url(url: str) -> str:
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
        if match_ is None:
            raise ValueError('Invalid Google Docs URL')
        doc_id = match_.group(1)
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
        response.raise_for_status()
        return response.text

# Функция подсчета токенов для модели эмбеддингов
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# функция подсчета токенов в сообщении модели
def num_tokens_from_messages(messages, model):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      if model in ["gpt-4-1106-preview", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-1106"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
          See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


# Функция разделения текста на чанки заданной длины (в токенах)
def split_text(text, max_count, chunk_overlap):

    # Функция для подсчета количества токенов во фрагменте для сплиттера RecursiveCharacterTextSplitter
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    num_levels = 3  # Число уровней заголовков, которые будем разделять сплиттером MarkdownHeaderTextSplitter

    headers_to_split_on = [
    (f"{'#' * i}", f"H{i}") for i in range(1, num_levels + 1)
    ]
    # сначала разделим с помощью MarkdownHeaderTextSplitter
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    fragments = markdown_splitter.split_text(text)


    # дальше будем делить делить каждый полученный чанк вторым сплиттером RecursiveCharacterTextSplitter
    # 1. для того, чтобы быть уверенным в размере полученного чанка
    # 2. база знаний размечена так, что заголовки не повторяются в тексте разделов, мы это исправим принудительно:

    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=num_tokens)

    source_chunks = []

    # Обработаем каждый фрагмент текста, полученный после MarkdownHeaderTextSplitter
    for fragment in fragments:
    # MarkdownHeaderTextSplitter сохранил иерархию заголовков в Метаданных - вытащим ее
      level = 0
      headers = ['','','','']
      for j in range(1, num_levels + 1):
        header_key = f'H{j}'
        if header_key in fragment.metadata: level, headers[j-1] = j, fragment.metadata[header_key]
      header_string = ' '.join([f"{'#' * i} {header}" for i, header in enumerate(headers[:level], start=1)])

    # каждый фрагмент будем разбивать на чанки с помощью RecursiveCharacterTextSplitter
    # допишем иерархию заголовков в конец чанка
    # унаследуем метаданные от первого сплиттера
    # добавим в метаданные размер чанка в токенах
      for i,chunk in enumerate(splitter.split_text(fragment.page_content)):
        mdata = fragment.metadata.copy()
        add_hierarchy = f'{header_string}: уровень {level} пункт {i+1}'
        new_chunk = ' '.join([chunk, f'\nРаздел: {add_hierarchy}'])
        mdata["len"] = num_tokens(new_chunk)
        doc = Document(page_content=new_chunk, metadata=mdata)
        source_chunks.append(doc)

    return source_chunks

# функция для очистки списка строк от повторений (для формирования более корректного точного саммари)
def list_cleaner(list_to_clean):
  filtered_list = [value.replace('"', '').strip() for value in list_to_clean if value.replace('"', '').strip() != '']
  text = ', '.join(filtered_list).replace('\n', ',').replace('-', ' ')
  phrases = text.split(',')
  return list(set(map(str.strip, text.split(','))))

# Запуск модели  суфлера при первом обращении клиента
def sufler(history_user):
  hello_completion = get_hello(MODEL,history_user[0],0.4,0)
  hello = hello_completion.choices[0].message.content
  try:
      hello_word = str(hello).split(': ')[1]+ ' '
  except:
      hello_word = str(hello)

  hello_word = hello_word.capitalize().rstrip(string.punctuation) + ', '

  if 'None' in hello_word: hello_word='Здравствуйте,'  # если пользователь не поздоровался, то мы поздороваемся нейтрально-формально

  return hello_word

# Функция удаления перехода на новую строку
def remove_newlines(text):
    cleaned_string = text.replace('\n', ' ')
    return cleaned_string

#фунция добавления переходов на новую строку для удобства чтения
# здесь в параметр text подается текст без переносов строки
# функция добавит переходы на новую строку при достижении max_len символов
def insert_newlines(text: str, max_len: int = 160) -> str:
      words = text.split()
      lines = []
      current_line = ""
      for word in words:
          if len(current_line + " " + word) > max_len:
              lines.append(current_line)
              current_line = ""
          current_line += f' {word}'
      lines.append(current_line)
      return "\n".join(lines)

#функция добавления переходов на новую строку для удобства чтения
# здесь в параметр text подается текст, содержащий переносы строки
# функция добавит переходы на новую строку при достижении max_len символов
def insert_newlines_n(text: str, max_len: int = 150) -> str:
    lines = text.splitlines()
    new_lines = []
    for line in lines:
        words = line.split()
        current_line = ""
        for word in words:
            if len(current_line + " " + word) > max_len:
                new_lines.append(current_line)
                current_line = ""
            current_line += f' {word}'
        new_lines.append(current_line)
    return "\n".join(new_lines)



In [33]:
#@title Ансамбль моделей для формирования ответа нейро-продажника

def get_seller_answer(history_user, history_manager, history_chat):
    output_router_list = []

    global needs_extractor
    global benefits_extractor
    global objection_detector
    global resolved_objection_detector
    global tariff_detector

    global summarized_dialog


    # 1. Выделим потребности из последнего сообщения клиента  и добавим в историю потребностей
    output_ne = extract_entity_from_statement(
                      name=name_needs_extractor,
                      system=system_prompt_needs_extractor,
                      instructions=instructions_needs_extractor,
                      question=history_user[-1],
                      history='',
                      temp=temperature_needs_extractor,
                      verbose=verbose_needs_extractor,
                      model=model_needs_extractor).choices[0].message.content
    try:
      output_needs_extractor = str(output_ne).split(':')[1]+ ''
    except:
      output_needs_extractor = str(output_ne)
    needs_extractor.append(output_needs_extractor)
    needs_extractor = list_cleaner(needs_extractor)

    # 2. Выделим названные менеджером преимущества, добавим в историю озвученных преимуществ
    output_be = extract_entity_from_statement(
                      name=name_benefits_extractor,
                      system=system_prompt_benefits_extractor,
                      instructions=instructions_benefits_extractor,
                      question='',
                      history=history_manager,
                      temp=temperature_benefits_extractor,
                      verbose=verbose_benefits_extractor,
                      model=model_benefits_extractor).choices[0].message.content
    try:
      output_benefits_extractor = str(output_be).split(':')[1]+ ''
    except:
      output_benefits_extractor = str(output_be)
    benefits_extractor.append(output_benefits_extractor)
    benefits_extractor = list_cleaner(benefits_extractor)

    #3. Выделим наличие возражений, добавим их в историю возражений
    output_obj = extract_entity_from_statement(
                      name=name_objection_detector,
                      system=system_prompt_objection_detector,
                      instructions=instructions_objection_detector,
                      question=history_user[-1],
                      history='',
                      temp=temperature_objection_detector,
                      verbose=verbose_objection_detector,
                      model=model_objection_detector).choices[0].message.content
    try:
        output_objection_detector = str(output_obj).split(':')[1]+ ''
    except:
        output_objection_detector = str(output_obj)
    objection_detector.append(output_objection_detector)
    objection_detector = list_cleaner(objection_detector)

    #4. Выделим отработанные менеджером возражения клиента, добавим их в историю отработанных возражений
    output_res = extract_entity_from_statement(
                      name=name_resolved_objection_detector,
                      system=system_prompt_resolved_objection_detector,
                      instructions=instructions_resolved_objection_detector,
                      question='',
                      history=history_manager,
                      temp=temperature_resolved_objection_detector,
                      verbose=verbose_resolved_objection_detector,
                      model=model_resolved_objection_detector).choices[0].message.content
    try:
        output_resolved_objection_detector = str(output_res).split(':')[1]+ ''
    except:
        output_resolved_objection_detector = str(output_res)
    resolved_objection_detector.append(output_resolved_objection_detector)
    resolved_objection_detector = list_cleaner(resolved_objection_detector)

    #5. Выделим названные менеджером тарифы и цены, добавим их в историю названных тарифов и цен
    output_tar = extract_entity_from_statement(
                      name=name_tariff_detector,
                      system=system_prompt_tariff_detector,
                      instructions=instructions_tariff_detector,
                      question='',
                      history=history_manager,
                      temp=temperature_tariff_detector,
                      verbose=verbose_tariff_detector,
                      model=model_tariff_detector).choices[0].message.content
    try:
        output_tariff_detector = str(output_tar).split(':')[1]+ ''
    except:
        output_tariff_detector = str(output_tar)
    tariff_detector.append(output_tariff_detector)
    tariff_detector = list_cleaner(tariff_detector)

    #6. Выделим ключи из последних сообщений клиента и менеджера (предыдущий вопрос+ответ)
    k = 2 if len(history_user)>1 else 1
    if history_manager and len(history_manager)>0:
      manager_list = history_manager[-1:]
    else:
      manager_list = []

    topicphrase_completion = get_topicphrase_questions(name=name_todo_base,
                                _user=history_user[-k:],
                                _manager = manager_list,
                                system=system_topicphrase_extractor,
                                instruction=instructions_topicphrase_extractor,
                                temp=temperature_todo_base,
                                verbose=verbose_base,
                                model=model_todo_base)
    topicphrase_answ = topicphrase_completion.choices[0].message.content

    # именно general_topic_phrase будем подавать в Langchain для similary search, преобразуем в строку-перечисление через запятую + добавляем текущий вопрос клиента
    general_topic_phrase = str(history_user[-1]+', ' + topicphrase_answ).replace('[','') .replace(']','').replace("'",'').replace("'",'')

    #7. Суммаризируем хронологию предыдущих сообщений диалога
    summarized_comp = summarize_dialog(summarized_dialog, history_chat, temp=0.1, verbose=verbose_router, model=MODEL)
    summarized_dialog = summarized_comp.choices[0].message.content

    #8. Создаем точное саммари с ключевыми моментами диалога
    tochnoe_summary = f'''
# 1. Выявлены Потребности: {', '.join(needs_extractor) if needs_extractor else 'потребностей не обнаружено'}
# 2. Расказанные Преимущества: {', '.join(benefits_extractor) if benefits_extractor else 'преимущества не были рассказаны'}
# 3. Возражения клиента: {', '.join(objection_detector) if objection_detector else 'возражений не обнаружено'}
# 4. Возражения клиента отработаны: {', '.join(resolved_objection_detector) if resolved_objection_detector else 'отработки не обнаружено'}
# 5. Конкретика - оговоренная конкретика - курсы, цены: {', '.join(tariff_detector) if tariff_detector else 'не обнаружено'}
'''
    #9. Запускаем Диспетчера
    output_router = user_question_router(name= name_router,
                                        system=system_prompt_router,
                                        instructions=instructions_router,
                                        question=history_user[-1],
                                        summary_history=summarized_dialog,
                                        summary_exact=tochnoe_summary,
                                        temp=temperature_router,
                                        verbose=verbose_router,
                                        model=model_router,
                                        needs_lst=needs_extractor).choices[0].message.content

    #10. По списку спецов из ответа Диспетчера запускаем спецов:
    output_spez = []
    try:
        output_router_fixed = (str(output_router).split(':')[1]+ '').replace("‘", '"').replace("'", '"')
    except:
        output_router_fixed = str(output_router).replace("‘", '"').replace("'", '"')

    try:
      output_router_list = json.loads(output_router_fixed)
    except:
      output_router_list = ['Zoom_Пуш', 'Спец_по_презентациям']
    print(f'{bcolors.RED}{output_router_list}{bcolors.ENDC}')
    try:
      for key_param in output_router_list:
          param = spez_config[key_param] | {'question': history_user[-1],
                                      'summary_history': summarized_dialog,
                                      'summary_exact': tochnoe_summary,
                                      'base_topicphrase': general_topic_phrase,
                                      'search_index': vectordb}
          spez_answer = spez_user_question(**param).choices[0].message.content
          try:
            answer = spez_answer.split(': ')[1]+ ' '
          except:
            answer = spez_answer
          answer = answer.lstrip('#3')

          output_spez.append(f'{param["name"]}: {answer}')

      if verbose: print(f"\n{bcolors.BGMAGENTA}Ответы спецов:{bcolors.ENDC}\n", '\n\n=========\n'.join(output_spez))
    except:
      if verbose: print(f'{bcolors.BGYELLOW}Ответ диспетчера либо не вызывает спецов либо имеет неверный формат:{bcolors.ENDC} {output_router}')


    #11. На основании предлоажения узких спецов запускаем страшего менеджера для подготовки комплексного ответа:
    output_senior = senior_answer(
                          name=name_senior,
                          system=system_prompt_senior,
                          instructions=instructions_senior,
                          question=history_user[-1],
                          output_spez=output_spez,
                          summary_history=summarized_dialog,
                          base_topicphrase=general_topic_phrase,
                          search_index=vectordb,
                          summary_exact=tochnoe_summary,
                          temp=temperature_senior,
                          verbose=verbose_senior,
                          k=num_fragments_base_senior,
                          model=model_senior,
                          spez_list=output_router_list).choices[0].message.content
    if verbose: print(f"\n{bcolors.BGMAGENTA}senior: {bcolors.ENDC} {output_senior}", )
    #12. Запускаем Стилиста:
    output_stilist = stilizator_answer(
                          name=name_stilist,
                          system=system_prompt_stilist,
                          instructions=instructions_stilist,
                          answers_content=output_senior,
                          temp=temperature_stilist,
                          verbose=verbose_stilist,
                          model=model_stilist).choices[0].message.content

    #13. контрольный выстрел по приветствиям:
    output_stilist_withouthello = del_hello(
                          name=name_stilist,
                          system=system_prompt_stilist1,
                          instructions=instructions_prompt_stilist1,
                          answers_content=output_stilist,
                          temp=temperature_stilist,
                          verbose=verbose_stilist,
                          model=model_stilist).choices[0].message.content

    return output_stilist_withouthello


# Подгружаем ключ API, Базу знаний


In [34]:
# запустите эту ячейку, если используете секретный ключ в колабе
from google.colab import userdata
key = userdata.get('Ni4ka')
os.environ["OPENAI_API_KEY"] = key

SecretNotFoundError: Secret Ni4ka does not exist.

In [ ]:
# @title Загружаем Базу Знаний, индексируем, сохраняем индексы
knowledge_db_txt = load_googledoc_by_url(knowledge_db_url)
# Разбиваем на чанки
docs = split_text(knowledge_db_txt, chunk_size, chunk_overlap)
vectordb = FAISS.from_documents(docs, OpenAIEmbeddings())
vectordb.save_local('UII_database_markdown')

In [ ]:
# @title Загружаем индексы Базы Знаний
vectordb = FAISS.load_local('/content/UII_database_markdown', OpenAIEmbeddings())

print('Индексирование Базы Знаний завершено!')

# Диалог

In [ ]:
#@title Запуск диалога
history_chat = []
history_user = []
history_manager = []

needs_extractor = []
benefits_extractor = []
objection_detector = []
resolved_objection_detector = []
tariff_detector = []

main_answer = ''
summarized_dialog = ''

while True:
    client_question = input(f'{bcolors.BGCYAN}Вопрос клиента:{bcolors.ENDC} ')
    history_user.append(client_question)
    history_chat.append(f"Клиент: {client_question}")
    if len(history_user) == 1: hello_word = sufler(history_user)  # запомнили приветствие
    if client_question.lower() in ['stop', 'стоп']:
        break
    without_hello = get_seller_answer(history_user, history_manager, history_chat)
    if len(history_chat) == 1 and 'None' not in hello_word:
        main_answer = f'{hello_word} меня зовут Василий, я менеджер отдела продаж УИИ. ' + without_hello
    else:
        main_answer = without_hello

 #   print(f'{bcolors.BGGREEN}Василий:{bcolors.ENDC}\n {insert_newlines(remove_newlines(main_answer), 160)}')
    print(f'{bcolors.BGGREEN}Василий:{bcolors.ENDC} {remove_newlines(main_answer)}')

    history_chat.append(f"Менеджер: {without_hello}")
    history_manager.append(without_hello) # не будем Василия хранить в истории, чтобы не путать gpt лишними именами

# Запуск сохранения
from datetime import datetime
text_file = f'dialog_{datetime.now().strftime("%d.%m.%Y_%H.%M.%S")}.txt'
with open(text_file, "w") as f:
    f.write(str('\n\n'.join(history_chat)))
